# MXNet/Gluon CNN example

In [1]:
import os
import sys
import numpy as np
import math
import mxnet as mx
from mxnet import nd, autograd
from mxnet import gluon
from common.params import *
from common.utils import *

In [2]:
# Force one-gpu
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
print("OS: ", sys.platform)
print("Python: ", sys.version)
print("MXNet: ", mx.__version__)
print("Numpy: ", np.__version__)
print("GPU: ", get_gpu_name())
print(get_cuda_version())
print("CuDNN Version ", get_cudnn_version())

OS:  linux
Python:  3.6.3 |Anaconda custom (64-bit)| (default, Oct 13 2017, 12:02:49) 
[GCC 7.2.0]
MXNet:  1.3.0
Numpy:  1.13.3
GPU:  ['Tesla V100-SXM2-16GB', 'Tesla V100-SXM2-16GB', 'Tesla V100-SXM2-16GB', 'Tesla V100-SXM2-16GB']
CUDA Version 9.1.85
CuDNN Version  7.1.3


## Build model

In [13]:
def build_model(n_classes=N_CLASSES):
    net = gluon.nn.HybridSequential()
    with net.name_scope():
        net.add(gluon.nn.Conv2D(channels=50, kernel_size=3, padding=1, activation='relu'))
        net.add(gluon.nn.Conv2D(channels=50, kernel_size=3, padding=1))
        net.add(gluon.nn.MaxPool2D(pool_size=2, strides=2))
        net.add(gluon.nn.Activation('relu'))
        # Equiv to gluon.nn.LeakyReLU(0)
        net.add(gluon.nn.Dropout(0.25))
        net.add(gluon.nn.Conv2D(channels=100, kernel_size=3, padding=1, activation='relu'))
        net.add(gluon.nn.Conv2D(channels=100, kernel_size=3, padding=1))
        net.add(gluon.nn.MaxPool2D(pool_size=2, strides=2))
        net.add(gluon.nn.Activation('relu'))
        net.add(gluon.nn.Dropout(0.25))
        net.add(gluon.nn.Flatten())
        net.add(gluon.nn.Dense(512, activation='relu'))
        net.add(gluon.nn.Dropout(0.25))
        net.add(gluon.nn.Dense(n_classes))
    return net

## Init model

In [8]:
def init_model(net, ctx, lr=LR, momentum=MOMENTUM):
    net.initialize(mx.init.Xavier(magnitude=2.24), ctx=ctx)
    trainer = gluon.Trainer(
        net.collect_params(), 
        'sgd',
        {'learning_rate': lr, 'momentum':momentum})
    criterion = gluon.loss.SoftmaxCrossEntropyLoss()
    return trainer, criterion

## Get data

In [17]:
%%time
# Data into format for library
x_train, x_test, y_train, y_test = cifar_for_library(channel_first=True)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
print(x_train.dtype, x_test.dtype, y_train.dtype, y_test.dtype)

Preparing train set...
Preparing test set...
(50000, 3, 32, 32) (10000, 3, 32, 32) (50000,) (10000,)
float32 float32 int32 int32
CPU times: user 776 ms, sys: 568 ms, total: 1.34 s
Wall time: 2.34 s


## Create model

In [30]:
%%time
ctx = mx.gpu()
net = build_model()

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 3.4 ms


In [31]:
%%time
trainer, criterion = init_model(net, ctx)

CPU times: user 8 ms, sys: 0 ns, total: 8 ms
Wall time: 4.42 ms


## Training Loop

In [32]:
%%time
net.hybridize()
for j in range(EPOCHS):
    train_loss = nd.zeros(1, ctx=ctx)
    for i, (data, target) in enumerate(yield_mb(x_train, y_train, BATCHSIZE, shuffle=True)):
        # Get samples
        data = nd.array(data).as_in_context(ctx)
        target = nd.array(target).as_in_context(ctx)
        with autograd.record():
            # Forwards
            output = net(data)
            # Loss
            loss = criterion(output, target)
        # Back-prop
        loss.backward()
        trainer.step(data.shape[0])
        train_loss += loss.mean()
    # Log    
    print('Epoch %3d: loss: %5.4f'%(j, train_loss.asscalar()/(i+1)))

Epoch   0: loss: 1.8582
Epoch   1: loss: 1.3819
Epoch   2: loss: 1.1333
Epoch   3: loss: 0.9515
Epoch   4: loss: 0.8145
Epoch   5: loss: 0.7097
Epoch   6: loss: 0.6174
Epoch   7: loss: 0.5324
Epoch   8: loss: 0.4575
Epoch   9: loss: 0.3964
CPU times: user 49 s, sys: 13.3 s, total: 1min 2s
Wall time: 37.2 s


## Evaluation loop

In [34]:
%%time
# Main evaluation loop: 453ms
n_samples = (y_test.shape[0]//BATCHSIZE)*BATCHSIZE
y_guess = np.zeros(n_samples, dtype=np.int)
y_truth = y_test[:n_samples]
c = 0
for data, target in yield_mb(x_test, y_test, BATCHSIZE):
    # Get samples
    data = nd.array(data).as_in_context(ctx)
    # Forwards
    output = net(data)
    pred = nd.argmax(output, axis=1)
    # Collect results
    y_guess[c*BATCHSIZE:(c+1)*BATCHSIZE] = pred.asnumpy()
    c += 1

CPU times: user 296 ms, sys: 48 ms, total: 344 ms
Wall time: 278 ms


In [35]:
print("Accuracy: ", 1.*sum(y_guess == y_truth)/len(y_guess))

Accuracy:  0.765324519231
